In [17]:
# INITIALIZE
import pandas as pd
#!/usr/bin/env python3
import os
from dotenv import load_dotenv
import typing
import fmpsdk
from helper import to_percentage


# Actual API key is stored in a .env file.  Not good to store API key directly in script.
load_dotenv()
apikey = os.environ.get("apikey")






In [35]:
# this block will be pasted at constant.py at the end of the development
from enum import Enum
class INCOMESTATEMENT(Enum):
    CALENDAR_YEAR='calendarYear'
    PERIOD='period'
    GROSSPROFIT_RATIO='grossProfitRatio'
    NETINCOME_RATIO='netIncomeRatio'
    GROSS_PROFIT='grossProfit'
    RD='researchAndDevelopmentExpenses'
    SGA_EXPENSE='sellingGeneralAndAdministrativeExpenses'
    DEPRECIATION='depreciationAndAmortization'
    OPERATING_INCOME='operatingIncome'
    INTEREST_EXPENSE='interestExpense'
    TAX='incomeTaxExpense'


class BALANCESHEET(Enum):
    CALENDAR_YEAR='calendarYear'
    PERIOD='period'
    LONGTERM_DEBT='longTermDebt'
    TOTAL_LIABILITIES='totalLiabilities'
    TOTAL_STOCKHOLDEREQUITIES='totalStockholdersEquity'
    TOTAL_DEBT='totalDebt'


class BALANCEGROWTH(Enum):
    CALENDAR_YEAR='calendarYear'
    RETAINEDEARNINGS_GROWTH='growthRetainedEarnings'

class INCOMEGROWTH(Enum):
    CALENDAR_YEAR='calendarYear'
    EBITDA_GROWTH='growthEBITDA'


class CASHFLOWSTATEMENT(Enum):
    CALENDAR_YEAR='calendarYear'
    PERIOD='period'
    CAPEX='capitalExpenditure'
    COMMONSTOCK_REPURCHASE='commonStockRepurchased'
    DIVIDEND_PAID='dividendsPaid'
    DEPRECIATION='depreciationAndAmortization'
    CHANGE_WORKING_CAP='changeInWorkingCapital'
    NET_INCOME='netIncome'
    FREE_CASH_FLOW='freeCashFlow'
    

class PERIOD(Enum):
    QUARTER='quarter'
    ANNUAL='annual'

class INTRINSICVALUATION(Enum):
    OPINCOME_AFTER_TAX='operating_income_after_tax'
    NET_CAPEX='net_capex'
    FCFF='free cash flow to firm'
    AUGMENTED_DIVIDEND='augmented dividend'
    RETENTION_RATIO='retention_ratio'
    EXPECTED_GROWTH_FCFE='expected growth in FCFE'
    EXPECTED_GROWTH_FCFF='expected growth in FCFF'
    REINVESTMENT_RATE='reinvestment rate'
    CALCULATE_TERMINAL_VAL='terminal value calculated'
    STOCK_INTRINSIC_VAL='intrinsic value per share'
    

class ADVANCED_DCF(Enum):
    CALENDAR_YEAR='year'
    RISK_FREE_RATE='riskFreeRate'
    WACC='wacc'
    TAX_RATE='taxRate'
    EBIT='ebit'
    LONG_TERM_GROWTH='longTermGrowthRate'
    DILUTED_SHARE_OUTSTANDING='dilutedSharesOutstanding'
    
    

class RATIO(Enum):
    CALENDAR_YEAR='calendarYear'
    PERIOD='period'
    RETURN_ON_EQUITY='returnOnEquity'
    RETURN_ON_CAPITAL='returnOnCapitalEmployed'
    LIABILITY_TO_SHAREHOLDER_EQUITY='liability_shareholder ratio'
    RD_GROSSPROFIT_RATIO='RD_grossProfit_Ratio'
    SGA_GROSSPROFIT_RATIO='SGA_grossProfit_Ratio'
    DEPRECIATION_GROSSPROFIT_RATIO='Depreciation_grossProfit_Ratio'
    INTEREST_EXP_OPINCOME_RATIO='interestExpense_operatingIncome_Ratio'
    DEBT_NETINCOME_RATIO='debt_ netIncome_ratio'
    CAPEX_GROSSPROFIT_RATIO='capex_grossProfit_rario'
    

In [36]:
# this block will be pasted at helper.py at the end of the development
import math
def to_percentage(val:float)->str:
    """
    convert decimal number to percentage
    Args:
        val (float): The decimal number grossProfitto convert
    Returns:
        str: percentage str of the data
    """
    return "{number:.2f} %".format(number=val*100)

def discount_terminal_value(fcff:float,
                            wacc:float,
                            risk_free_rate:float,
                            expected_growth_in_n:float,
                            reinvestment_rate:float,
                            n:int=5)->float:

    """
    
    Args:
        fcff (float): Free cash flow to firm, come from EBIT *(1-Tax rate) - capital expenditure - change in working capital
        wacc (float): cost of capital
        risk_free_rate (float): interest rate of risk-free investment
        expected_growth_in_n (float): expected growth within estimated period = return on capital * reinvestment_rate
        reinvestment_rate (float): portion of after tax income that get reinvested = (change in working capital + capital expenditure) / EBIT 

    """
    
    # fcff at year n
    fcff_end_period=fcff*math.pow(1+expected_growth_in_n,n)
    
    # reinvestment rate at stable grow
    reinvest_stable_rate=risk_free_rate/wacc
    # operating income after tax in year n+1
    opIncome_end_period=((fcff_end_period)/(1-reinvestment_rate))*(1+risk_free_rate)
    fcff_over_end=opIncome_end_period*(1-reinvest_stable_rate)



    # terminal value at year n+1
    terminal_val_end_period=(fcff_over_end)/(wacc-risk_free_rate)
    # print(f'reinvestment at stable: {reinvest_stable_rate}')
    # print(f'FCFF at end of the period: {fcff_end_period}')
    # print(f'operating income after tax at year n+1: {opIncome_end_period}')
    # print(f'Terminal value at year n+1 {terminal_val_end_period}')

    # calculate total fcff within the period
    cf_sum=0
    for i in range(1,n+1):
        discounted_rate=math.pow(1+wacc,i)
        growth_rate=math.pow(1+expected_growth_in_n,i)
        discounted_cf=fcff*growth_rate/discounted_rate
        # print(f'FCFF at year {i}: {fcff*growth_rate}')
        cf_sum+=discounted_cf
    
    precent_terminal=terminal_val_end_period/(math.pow(1+wacc,n))
    
    return precent_terminal+cf_sum
    



In [37]:
# Get data from SDK
symbol: str = input("Please enter stock symbol ")
period:str=input("Enter 'quarter' and 'annual'")
period=period.strip().lower()
symbol=symbol.strip().upper()
# financial statement
income_statement=fmpsdk.income_statement(apikey=apikey,period=period,symbol=symbol)
balance_sheet=fmpsdk.balance_sheet_statement(apikey=apikey,period=period,symbol=symbol)
cashflow_statement=fmpsdk.cash_flow_statement(apikey=apikey,period=period,symbol=symbol)
ratio_data=fmpsdk.financial_ratios(apikey=apikey,symbol=symbol,period=period)


# growth data
if period==PERIOD.ANNUAL.value:
    balance_growth=fmpsdk.balance_sheet_statement_growth(apikey=apikey,symbol=symbol)
    income_growth=fmpsdk.income_statement_growth(apikey=apikey,symbol=symbol)



In [38]:
# FOR THOSE NOT IN SDK, EXTRACT USING URL REQUEST
import requests

if period==PERIOD.ANNUAL.value:
    # advanced discounted cash flow data
    resp=requests.get(f'https://financialmodelingprep.com/api/v4/advanced_discounted_cash_flow',
                    params={
                        'symbol':symbol,
                        'apikey':apikey
                        })
    advance_dcf_data=resp.json()
    advance_dcf_frame=pd.DataFrame(data=advance_dcf_data)
    advance_dcf_frame=advance_dcf_frame.set_index(keys=ADVANCED_DCF.CALENDAR_YEAR.value,drop=True)
    # for capex, please refer to cashflow statement
    if CASHFLOWSTATEMENT.CAPEX.value in advance_dcf_frame.columns:
        advance_dcf_frame=advance_dcf_frame.drop(labels=CASHFLOWSTATEMENT.CAPEX.value,axis='columns')


In [40]:
# FINANCE RATIO
ratio_frame=pd.DataFrame(data=ratio_data)
if period==PERIOD.ANNUAL.value:
    ratio_frame=ratio_frame.set_index(keys=RATIO.CALENDAR_YEAR.value)
elif period==PERIOD.QUARTER.value:
    ratio_frame=ratio_frame.set_index(keys=[RATIO.CALENDAR_YEAR.value,RATIO.PERIOD.value])

symbol        date  currentRatio  quickRatio  cashRatio  \
calendarYear period                                                           
2023         Q3        TSM  2023-09-30      1.869418    0.000000   0.000000   
             Q2        TSM  2023-06-30      2.417235    2.073166   1.574591   
             Q1        TSM  2023-03-31      2.285821    2.018641   1.586587   
2022         Q4        TSM  2022-12-31      2.174156    1.925288   1.422131   
             Q3        TSM  2022-09-30      2.494618    2.205203   1.605106   
             Q2        TSM  2022-06-30      2.254820    1.981480   1.482641   
             Q1        TSM  2022-03-31      2.092970    1.835946   1.399483   
2021         Q4        TSM  2021-12-31      2.119163    1.850427   1.404347   
             Q3        TSM  2021-09-30      2.090595    1.795290   1.302294   
             Q2        TSM  2021-06-30      1.881720    1.612074   1.134065   

                     daysOfSalesOutstanding  daysOfInventoryOutstanding  \
calendarYear period                                                       
2023         Q3                    0.000000                   94.318445   
             Q2                   35.754917                   95.585000   
             Q1                   26.209834                   87.542846   
2022         Q4                   33.294636                   84.212735   
             Q3                   38.401010                   80.983773   
             Q2                   37.841813                   89.491820   
             Q1                   39.125819                   82.666074   
2021         Q4                   40.734864                   83.751147   
             Q3                   40.648875                   81.264988   
             Q2                   43.524537                   82.493267   

                     operatingCycle  daysOfPayablesOutstanding  \
calendarYear period                                              
2023         Q3           94.318445                   0.000000   
             Q2          131.339917                  77.389696   
             Q1          113.752680                  82.522470   
2022         Q4          117.507371                 102.197801   
             Q3          119.384783                  81.574230   
             Q2          127.333633                  89.934650   
             Q1          121.791894                  70.872496   
2021         Q4          124.486010                  83.718835   
             Q3          121.913863                  75.854512   
             Q2          126.017803                  71.086806   

                     cashConversionCycle  ...  priceToSalesRatio  \
calendarYear period                       ...                      
2023         Q3                94.318445  ...          26.621459   
             Q2                53.950221  ...          33.871240   
             Q1                31.230210  ...          28.785671   
2022         Q4                15.309570  ...          18.970171   
             Q3                37.810552  ...          18.341335   
             Q2                37.398983  ...          23.609402   
             Q1                50.919398  ...          31.562823   
2021         Q4                40.767175  ...          39.376372   
             Q3                46.059351  ...          38.765302   
             Q2                54.930997  ...          46.683527   

                     priceEarningsRatio  priceToFreeCashFlowsRatio  \
calendarYear period                                                  
2023         Q3               17.245059                 213.973916   
             Q2               22.396549                -195.551247   
             Q1               17.683928                 177.600470   
2022         Q4               12.698123                  79.067240   
             Q3               10.009991                  77.524299   
             Q2               13.300940                 106.957613   


In [41]:
# CASHFLOW STATEMENT
cashflow_columns=[
    CASHFLOWSTATEMENT.CALENDAR_YEAR.value,
    CASHFLOWSTATEMENT.CAPEX.value,
    CASHFLOWSTATEMENT.COMMONSTOCK_REPURCHASE.value,
    CASHFLOWSTATEMENT.DIVIDEND_PAID.value,
    CASHFLOWSTATEMENT.DEPRECIATION.value,
    CASHFLOWSTATEMENT.CHANGE_WORKING_CAP.value,
    CASHFLOWSTATEMENT.NET_INCOME.value,
    CASHFLOWSTATEMENT.FREE_CASH_FLOW.value
]

if period==PERIOD.QUARTER.value:
    cashflow_columns.append('period')

cashflow_frame=pd.DataFrame(data=cashflow_statement,columns=cashflow_columns)

if period==PERIOD.ANNUAL.value:
    cashflow_frame=cashflow_frame.set_index(keys=CASHFLOWSTATEMENT.CALENDAR_YEAR.value)
elif period==PERIOD.QUARTER.value:
    cashflow_frame=cashflow_frame.set_index(keys=[CASHFLOWSTATEMENT.CALENDAR_YEAR.value,CASHFLOWSTATEMENT.PERIOD.value])

capitalExpenditure  commonStockRepurchased  \
calendarYear period                                               
2023         Q3           -226623760000                       0   
             Q2           -250534000000                       0   
             Q1           -302804977000                       0   
2022         Q4           -336800000000                  -34000   
             Q3           -267635828000                       0   
             Q2           -220945293000                       0   
             Q1           -262717722000              -871566000   
2021         Q4           -235556000000                       0   
             Q3           -188640000000                       0   
             Q2           -170513981000                       0   

                     dividendsPaid  depreciationAndAmortization  \
calendarYear period                                               
2023         Q3       -71308546000                 145904387000   
             Q2       -71309000000                 123939000000   
             Q1       -71308546000                 110322641000   
2022         Q4       -71300000000                 107581924000   
             Q3       -71308546000                 105345666000   
             Q2       -71308547000                 113224647000   
             Q1       -71308546000                 111102063000   
2021         Q4       -71308000000                 110663000000   
             Q3       -64826000000                 107027000000   
             Q2       -64825949000                 103805182000   

                     changeInWorkingCapital     netIncome  freeCashFlow  
calendarYear period                                                      
2023         Q3                -60060562000  210999939000   68021516000  
             Q2                -50085000000  214675000000  -83286000000  
             Q1                 43998868000  244274931000   82439768000  
2022         Q4                 55492398000  334551356000  150080000000  
             Q3                 11411806000  316690867000  145062339000  
             Q2                 22303142000  265998281000  117904136000  
             Q1                 33301154000  226831696000  109451966000  
2021         Q4                 84527000000  184819000000  142644000000  
             Q3                 68705000000  173852000000  130066000000  
             Q2                 -7363705000  149375511000   16925513000

In [42]:
# BALANCE SHEET
balance_columns=[BALANCESHEET.CALENDAR_YEAR.value,
                BALANCESHEET.LONGTERM_DEBT.value,
                BALANCESHEET.TOTAL_LIABILITIES.value,
                BALANCESHEET.TOTAL_STOCKHOLDEREQUITIES.value]

if period==PERIOD.QUARTER.value:
    balance_columns.append('period')
balance_frame=pd.DataFrame(data=balance_sheet,columns=balance_columns)

# ratio
balance_frame[RATIO.LIABILITY_TO_SHAREHOLDER_EQUITY.value]=balance_frame[BALANCESHEET.TOTAL_LIABILITIES.value]/balance_frame[BALANCESHEET.TOTAL_STOCKHOLDEREQUITIES.value]

# set index
if period==PERIOD.ANNUAL.value:
    balance_frame=balance_frame.set_index(keys=BALANCESHEET.CALENDAR_YEAR.value)
elif period==PERIOD.QUARTER.value:
    balance_frame=balance_frame.set_index(keys=[BALANCESHEET.CALENDAR_YEAR.value, BALANCESHEET.PERIOD.value])


longTermDebt  totalLiabilities  totalStockholdersEquity  \
calendarYear period                                                            
2023         Q3                 0     2111740000000            3356102256000   
             Q2      901710000000     1943997000000            3188750000000   
             Q1      884770024000     1952946750000            3078233958000   
2022         Q4      868860583000     2004290011000            2945653195000   
             Q3      875624002000     1890985607000            2737974306000   
             Q2      779719008000     1835479210000            2496353192000   
             Q1      653803868000     1671207913000            2314430096000   
2021         Q4      634143900000     1554770250000            2168286553000   
             Q3      484258621000     1253991574000            2076020062000   
             Q2      434666600000     1110653900000            1979870900000   

                     liability_shareholder ratio  
calendarYear period                               
2023         Q3                         0.629224  
             Q2                         0.609642  
             Q1                         0.634437  
2022         Q4                         0.680423  
             Q3                         0.690651  
             Q2                         0.735264  
             Q1                         0.722082  
2021         Q4                         0.717050  
             Q3                         0.604036  
             Q2                         0.560973

In [43]:
# BALANCE GROWTH
# sheet analysis only happend once a year
if period==PERIOD.ANNUAL.value:
    balance_growth_columns=[BALANCEGROWTH.CALENDAR_YEAR.value,
                            BALANCEGROWTH.RETAINEDEARNINGS_GROWTH.value]
    balance_growth_frame=pd.DataFrame(data=balance_growth,columns=balance_growth_columns)
    balance_growth_frame[BALANCEGROWTH.RETAINEDEARNINGS_GROWTH.value]=balance_growth_frame[BALANCEGROWTH.RETAINEDEARNINGS_GROWTH.value].map(to_percentage)
    balance_growth_frame=balance_growth_frame.set_index(BALANCEGROWTH.CALENDAR_YEAR.value)

,growthRetainedEarnings
calendarYear,
2022,-26.52 %
2021,23.31 %
2020,-8.55 %
2019,28.51 %
2018,26.58 %
2017,10.98 %
2016,5.49 %
2015,12.34 %
2014,-19.25 %


In [44]:
# INCOME GROWTH

if period==PERIOD.ANNUAL.value:
    income_growth_columns=[INCOMEGROWTH.CALENDAR_YEAR.value,INCOMEGROWTH.EBITDA_GROWTH.value]
    income_growth_frame=pd.DataFrame(data=income_growth,columns=income_growth_columns)
    income_growth_frame[INCOMEGROWTH.EBITDA_GROWTH.value]=income_growth_frame[INCOMEGROWTH.EBITDA_GROWTH.value].map(to_percentage)
    income_growth_frame=income_growth_frame.set_index(keys=INCOMEGROWTH.CALENDAR_YEAR.value)

    

,growthEBITDA
calendarYear,
2022,-12.22 %
2021,66.26 %
2020,47.36 %
2019,31.82 %
2018,74.26 %
2017,47.56 %
2016,-6.36 %
2015,-5.16 %
2014,-32.74 %


In [45]:
# INCOME STATEMENT
# from constant import INCOMESTATEMENT


# selecting fields from original data structure and import them into pandas.DataFrame
income_columns=[INCOMESTATEMENT.CALENDAR_YEAR.value,
                INCOMESTATEMENT.GROSSPROFIT_RATIO.value,
                INCOMESTATEMENT.NETINCOME_RATIO.value,
                INCOMESTATEMENT.GROSS_PROFIT.value,
                INCOMESTATEMENT.RD.value,
                INCOMESTATEMENT.SGA_EXPENSE.value,
                INCOMESTATEMENT.DEPRECIATION.value,
                INCOMESTATEMENT.INTEREST_EXPENSE.value,
                INCOMESTATEMENT.OPERATING_INCOME.value,
                INCOMESTATEMENT.TAX.value
                ]
if period==PERIOD.QUARTER.value:
    income_columns.append('period')
income_frame=pd.DataFrame(data=income_statement,columns=income_columns)

# extract necessary ratio from income statement
income_frame[RATIO.RD_GROSSPROFIT_RATIO.value]=(income_frame[INCOMESTATEMENT.RD.value]/income_frame[INCOMESTATEMENT.GROSS_PROFIT.value]).map(to_percentage)
income_frame[RATIO.SGA_GROSSPROFIT_RATIO.value]=(income_frame[INCOMESTATEMENT.SGA_EXPENSE.value]/income_frame[INCOMESTATEMENT.GROSS_PROFIT.value]).map(to_percentage)
income_frame[RATIO.DEPRECIATION_GROSSPROFIT_RATIO.value]=(income_frame[INCOMESTATEMENT.DEPRECIATION.value]/income_frame[INCOMESTATEMENT.GROSS_PROFIT.value]).map(to_percentage)
income_frame[INCOMESTATEMENT.GROSSPROFIT_RATIO.value]=income_frame[INCOMESTATEMENT.GROSSPROFIT_RATIO.value].map(to_percentage)
income_frame[INCOMESTATEMENT.NETINCOME_RATIO.value]=income_frame[INCOMESTATEMENT.NETINCOME_RATIO.value].map(to_percentage)
income_frame[RATIO.INTEREST_EXP_OPINCOME_RATIO.value]=(income_frame[INCOMESTATEMENT.INTEREST_EXPENSE.value]/income_frame[INCOMESTATEMENT.OPERATING_INCOME.value]).map(to_percentage)

income_columns_todrop=[INCOMESTATEMENT.DEPRECIATION.value]
income_frame=income_frame.drop(labels=income_columns_todrop,axis='columns')

if period==PERIOD.ANNUAL.value:
    income_frame=income_frame.set_index(keys=INCOMESTATEMENT.CALENDAR_YEAR.value)
elif period==PERIOD.QUARTER.value:
    income_frame=income_frame.set_index(keys=[INCOMESTATEMENT.CALENDAR_YEAR.value,INCOMESTATEMENT.PERIOD.value])

grossProfitRatio netIncomeRatio   grossProfit  \
calendarYear period                                                 
2023         Q3              54.26 %        38.59 %  296643000000   
             Q2              54.11 %        37.81 %  260200000000   
             Q1              56.33 %        40.69 %  286500392000   
2022         Q4              62.22 %        37.35 %  389183047000   
             Q3              60.43 %        45.81 %  370498717000   
             Q2              59.06 %        44.38 %  315467994000   
             Q1              55.63 %        41.28 %  273203166000   
2021         Q4              52.65 %        37.93 %  230754823000   
             Q3              51.32 %        37.69 %  212746195000   
             Q2              50.03 %        36.10 %  186196818000   

                     researchAndDevelopmentExpenses  \
calendarYear period                                   
2023         Q3                                   0   
             Q2                         41665000000   
             Q1                         39157036000   
2022         Q4                         44587023000   
             Q3                         42977507000   
             Q2                         39649130000   
             Q1                         36048540000   
2021         Q4                         32239000000   
             Q3                         30866634000   
             Q2                         30873376000   

                     sellingGeneralAndAdministrativeExpenses  interestExpense  \
calendarYear period                                                             
2023         Q3                                            0                0   
             Q2                                  16530000000       3005155000   
             Q1                                  16152308000       2963686000   
2022         Q4                                  19948307000       3334985000   
             Q3                                  17208955000       3370018000   
             Q2                                  13725388000       2904231000   
             Q1                                  12562650000       2140766000   
2021         Q4                                  15639000000                0   
             Q3                                  10783282000       1334063000   
             Q2                                   9710047000       1134399000   

                     operatingIncome  incomeTaxExpense RD_grossProfit_Ratio  \
calendarYear period                                                           
2023         Q3         228065000000       30940000000               0.00 %   
             Q2         201958000000       32958000000              16.01 %   
             Q1         231238157000       37325895000              13.67 %   
2022         Q4         324644923000       62278689000              11.46 %   
             Q3         310324214000       35722460000              11.60 %   
             Q2         262123749000       28818029000              12.57 %   
             Q1         223790118000       23958322000              13.19 %   
2021         Q4         182771954000       18446000000              13.97 %   
             Q3         171003548000       17372485000              14.51 %   
             Q2         145667205000       14909359000              16.58 %   

                    SGA_grossProfit_Ratio Depreciation_grossProfit_Ratio  \
calendarYear period                                                        
2023         Q3                    0.00 %                         0.00 %   
             Q2                    6.35 %                        47.63 %   
             Q1                    5.64 %                        43.75 %   
2022         Q4                    5.13 %                      -100.00 %   
             Q3                    4.64 %                        30.78 %   
             Q2                    4.35 %                        37.82 %   


In [46]:
# RATIO TABLE
joinkeys=['calendarYear','period'] if period==PERIOD.QUARTER.value else ['calendarYear']
table1=pd.merge(left=income_frame,right=balance_frame,left_index=True,right_index=True)
table1=pd.merge(left=table1,right=cashflow_frame,left_index=True,right_index=True)
if period==PERIOD.ANNUAL.value:
    table1=pd.merge(left=table1,right=balance_growth_frame,left_index=True,right_index=True)
    table1=pd.merge(left=table1,right=income_growth_frame,left_index=True,right_index=True)
#ratio
table1[RATIO.DEBT_NETINCOME_RATIO.value]=(table1[BALANCESHEET.LONGTERM_DEBT.value]/table1[CASHFLOWSTATEMENT.NET_INCOME.value]).round(2)
table1[RATIO.CAPEX_GROSSPROFIT_RATIO.value]=(table1[CASHFLOWSTATEMENT.CAPEX.value].abs()/table1[INCOMESTATEMENT.GROSS_PROFIT.value]).map(to_percentage)

table1_columns=[
    INCOMESTATEMENT.GROSSPROFIT_RATIO.value,
    INCOMESTATEMENT.NETINCOME_RATIO.value,
    RATIO.RD_GROSSPROFIT_RATIO.value,
    RATIO.SGA_GROSSPROFIT_RATIO.value,
    RATIO.DEPRECIATION_GROSSPROFIT_RATIO.value,
    RATIO.INTEREST_EXP_OPINCOME_RATIO.value,
    RATIO.LIABILITY_TO_SHAREHOLDER_EQUITY.value,
    RATIO.DEBT_NETINCOME_RATIO.value,
    RATIO.CAPEX_GROSSPROFIT_RATIO.value,
    CASHFLOWSTATEMENT.COMMONSTOCK_REPURCHASE.value,
    CASHFLOWSTATEMENT.DIVIDEND_PAID.value,
]
if period==PERIOD.ANNUAL.value:
    table1_columns.append(BALANCEGROWTH.RETAINEDEARNINGS_GROWTH.value)
    table1_columns.append(INCOMEGROWTH.EBITDA_GROWTH.value)


table1=table1[table1_columns]

table1.style.set_caption(f'Table 1: stats for {symbol}')




## ratio table explain

| ratio                                     | description                                                                                       | bias                                                                                                                                 |
| ----------------------------------------- | ------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------ |
| gross profit ratio                        | How many money a company makes out of gross revenue after subtracting cost of good sold and labor | higher mean that company has more freedom to adjust the price                                                                        |
| net income ratio                          | net income is gross profit subtract SGA, operating expense, interest and tax expense              | consistently above 20 % is good. If net income ratio is below 10% means that company is facing competition                           |
| RD and gross profit ratio                 | it shows whether the company needs to spend a lot of money to keep up with the competition        |                                                                                                                                      |
| SGA and gross profit ratio                | the ratio tells us how quickly the company is able to cut the cost                                | If the company keep the ratio consistent, it tells us that the company is able to quickly cut the cost when the time is bad for them |
| Depreciation and gross profit ratio       | depreciation is expense spread over the years                                                     | company with durable competitive advantage tends to have lower depreciation and gross profit ratio                                   |
| total liability to shareholder equity     | the ratio tells us how many dolloar in debt for each dollor of shareholder equity                 | company with durable competitive advantage tend to keep this ratio around or below one                                               |
| capital expenditure to gross profit ratio | capital expenditure is the expense that company spend to continue its operation                   | company with durable competitive advantage will keep it consistently below 25 % |


<!-- TODO: write an MD file to explain each ratio -->

In [34]:
# DISCOUNT CASH FLOW - GORDON GROWTH MODEL

if period==PERIOD.ANNUAL.value:


   
    table2=pd.merge(left=income_frame,right=cashflow_frame,left_index=True,right_index=True)
    table2=pd.merge(left=table2,right=ratio_frame,left_index=True,right_index=True)
    table2=pd.merge(left=table2,right=balance_frame,left_index=True,right_index=True)
    table2=pd.merge(left=table2,right=advance_dcf_frame,left_index=True,right_index=True)

    # data transform
    table2[ADVANCED_DCF.TAX_RATE.value]=table2[ADVANCED_DCF.TAX_RATE.value]/100
    table2[ADVANCED_DCF.WACC.value]=table2[ADVANCED_DCF.WACC.value]/100
    table2[ADVANCED_DCF.RISK_FREE_RATE.value]=table2[ADVANCED_DCF.RISK_FREE_RATE.value]/100

    # operations
    table2[INTRINSICVALUATION.OPINCOME_AFTER_TAX.value]=table2[ADVANCED_DCF.EBIT.value]*(1-table2[ADVANCED_DCF.TAX_RATE.value])
    table2[INTRINSICVALUATION.NET_CAPEX.value]=table2[CASHFLOWSTATEMENT.CAPEX.value].abs()-table2[CASHFLOWSTATEMENT.DEPRECIATION.value]
    table2[INTRINSICVALUATION.FCFF.value]=table2[INTRINSICVALUATION.OPINCOME_AFTER_TAX.value]-table2[INTRINSICVALUATION.NET_CAPEX.value]-table2[CASHFLOWSTATEMENT.CHANGE_WORKING_CAP.value]
    table2[INTRINSICVALUATION.AUGMENTED_DIVIDEND.value]=table2[CASHFLOWSTATEMENT.DIVIDEND_PAID.value]+table2[CASHFLOWSTATEMENT.COMMONSTOCK_REPURCHASE.value]
    table2[INTRINSICVALUATION.RETENTION_RATIO.value]=(table2[CASHFLOWSTATEMENT.NET_INCOME.value]-table2[INTRINSICVALUATION.AUGMENTED_DIVIDEND.value].abs())/table2[CASHFLOWSTATEMENT.NET_INCOME.value]
    table2[INTRINSICVALUATION.EXPECTED_GROWTH_FCFE.value]=table2[INTRINSICVALUATION.RETENTION_RATIO.value]*table2[RATIO.RETURN_ON_EQUITY.value]
    table2[INTRINSICVALUATION.REINVESTMENT_RATE.value]=(table2[CASHFLOWSTATEMENT.CHANGE_WORKING_CAP.value]+table2[CASHFLOWSTATEMENT.CAPEX.value].abs())/table2[INCOMESTATEMENT.OPERATING_INCOME.value]
    table2[INTRINSICVALUATION.EXPECTED_GROWTH_FCFF.value]=table2[INTRINSICVALUATION.REINVESTMENT_RATE.value]*table2[RATIO.RETURN_ON_CAPITAL.value]
    
    table2[INTRINSICVALUATION.CALCULATE_TERMINAL_VAL.value]=table2.apply(lambda x: discount_terminal_value(
        fcff=x[INTRINSICVALUATION.FCFF.value],
        wacc=x[ADVANCED_DCF.WACC.value],
        risk_free_rate=x[ADVANCED_DCF.RISK_FREE_RATE.value],
        expected_growth_in_n=x[INTRINSICVALUATION.EXPECTED_GROWTH_FCFF.value],
        reinvestment_rate=x[INTRINSICVALUATION.REINVESTMENT_RATE.value]
    ),axis='columns')

    table2[INTRINSICVALUATION.STOCK_INTRINSIC_VAL.value]=table2[INTRINSICVALUATION.CALCULATE_TERMINAL_VAL.value]/table2[ADVANCED_DCF.DILUTED_SHARE_OUTSTANDING.value]
    
    table2_col=[# ADVANCED_DCF.EBIT.value,
                # ADVANCED_DCF.TAX_RATE.value,
                # INTRINSICVALUATION.OPINCOME_AFTER_TAX.value,
                # INTRINSICVALUATION.NET_CAPEX.value,
                # CASHFLOWSTATEMENT.CHANGE_WORKING_CAP.value,
                INTRINSICVALUATION.FCFF.value,

                
                # INTRINSICVALUATION.AUGMENTED_DIVIDEND.value,
                # CASHFLOWSTATEMENT.NET_INCOME.value,
                # INTRINSICVALUATION.RETENTION_RATIO.value,
                # RATIO.RETURN_ON_EQUITY.value,
                # INTRINSICVALUATION.EXPECTED_GROWTH_FCFE.value,
                INTRINSICVALUATION.REINVESTMENT_RATE.value,
                RATIO.RETURN_ON_CAPITAL.value,
                INTRINSICVALUATION.EXPECTED_GROWTH_FCFF.value,
                ADVANCED_DCF.WACC.value,
                ADVANCED_DCF.RISK_FREE_RATE.value,
                INTRINSICVALUATION.CALCULATE_TERMINAL_VAL.value,
                ADVANCED_DCF.DILUTED_SHARE_OUTSTANDING.value,
                INTRINSICVALUATION.STOCK_INTRINSIC_VAL.value
                
                
                ]

    table2=table2[table2_col]
    

table2.style.set_caption(f'Table 2: Intrinsic stock value for {symbol}')

,free cash flow to firm,reinvestment rate,returnOnCapitalEmployed,expected growth in FCFF,wacc,riskFreeRate,terminal value calculated,dilutedSharesOutstanding,intrinsic value per share
2022,116796680.000000,0.207418,1.054113,0.218641,0.095300,0.038400,3551121727.521072,11276000,314.927432
2021,228916980.000000,-0.247118,0.946375,-0.233866,0.095200,0.038400,779079109.597235,11276000,69.091798
2020,73774420.000000,0.242019,0.814398,0.197100,0.095200,0.038400,2141909273.204677,11276000,189.952933
2019,70475700.000000,0.017048,0.789823,0.013465,0.095300,0.038400,810689093.320104,11276000,71.895095
2018,50371840.000000,0.064767,0.632990,0.040997,0.095300,0.038400,671900135.186102,11276000,59.586745


In [112]:
discount_terminal_value(2510,0.0676,0.03,0.075,0.3)

69367.70266727956